In [1]:
import pandas as pd
import numpy as np
import random

import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from linearmodels.iv import IVGMM
from linearmodels import PanelOLS

import matplotlib.pyplot as plt
import seaborn as sns

from linearmodels.datasets import card

In [ ]:
sns.set(rc={'figure.figsize':(18,15)})
sns.set_style("white")

# Do political differences between countries affect their trade flows?
---

## 1. Introduction
- Let's test whether political differences (as measured by Polity scores) affect bilateral tradeflows between countries from 1960-2006.
- Unit of interest is the annual trade relationship between two countries (e.g. NZL-AUS).
- I will first estimate the gravity model of trade and a modified gravity model by including political difference as a explanatory variable.
- I will then look at **three different identification strategies to estimate the effect of political differences on trade flows**:
    - *Fixed Effects*
    - *Arellano-Bond Instrumental Variables*
    - *Anderson-Hsiao Instrumental Variables*
- Physical distance between countries are included controls.

## 2. Summary Statistics
---

In [ ]:
# Filter tradelinks with highest average trade volume over time

links = data.groupby('tradelink')['flow'].agg('mean').nlargest(n=15).index
largest_tradelinks = data[data['tradelink'].isin(links)]

### 2.1 Trade flows between 1990-2006 for top 10 tradelinks with highest trade flows

In [ ]:
# Tradeflows across top 10 tradelinks with highest average total trade value over time

ax = sns.lineplot(x='year',y='total_gdp', hue='tradelink', data=largest_tradelinks.query('year >= 1990'), palette='Set2')
ax.set(xlabel='Year', ylabel='Trade Flow (current US$ million)')

### 2.2 Jointplot of Polity Distance and Trade Flows (current USD million)

In [ ]:
# Joint plot of polity distance and tradeflow

grid = sns.jointplot(x='polity_dist', y='flow', kind='reg', color='#053259', x_jitter=0.5, y_jitter=100000, data=largest_tradelinks)
grid.fig.set_figwidth(12)
grid.fig.set_figheight(8)
grid.set_axis_labels(xlabel='Polity Distance', ylabel='Trade Flow (current US$ million)')

In [ ]:
# Change in polity distance over time

data['wto'] = data['gatt_o'] + data['gatt_d']
ax = sns.lineplot(x='year', y='polity_dist', hue='wto', palette='Set2', data=data)
ax.legend(['Trade between non-WTO countries', 'Trade between one non-WTO and one WTO country', 'Trade between WTO-countries'])
ax.set(xlabel='Year', ylabel='Polity')

In [ ]:
data.groupby('year')['wto'].value_counts()

In [ ]:
# Relationship between distance and flow

grid = sns.jointplot(x='distw',y='flow', kind='reg',color='#053259', data=largest_tradelinks)
grid.fig.set_figwidth(12)
grid.fig.set_figheight(8)
grid.set_axis_labels(xlabel='Distance (km)', ylabel='Trade Flow (current US$ million)')

## 3. Results
---

In [ ]:
# Set entity and time indexes
polity_data = data.set_index(['tradelink', 'year']).dropna(subset=['total_gdp', 'polity_dist'])

# Include a year column
polity_data['year'] = polity_data.index.get_level_values('year').astype('str')

# Include a tradelink column
polity_data['tradelink'] = polity_data.index.get_level_values('tradelink').astype('str')

In [ ]:
# Write function that returns original dataframe with additional lags for a specified col
# Patsy in Python does not have a lag operator to use in formulas

def get_lags(data, col, lag, level=0):
    for i in range(lag):
        data[f'{col}_{i+1}'] = data.groupby(level=level)[col].shift(i+1)
    return data

In [ ]:
# Create 30 lags of total_gdp
n_lags = 30
get_lags(polity_data, 'total_gdp', n_lags)
 
# Create 30 lags of polity_dist
get_lags(polity_data, 'polity_dist', n_lags)

# Set gdp_lags patsy formula
n_gdp_lags = 5
gdp_lags = '+'.join([f'total_gdp_{i}' for i in range(1, n_gdp_lags)])

### 3.1 Fixed Effects
---

In [ ]:
# Two-way fixed effects
# Note: we removed the distw covariate because it is captured in EntityEffects

formula_fe = f'flow ~ polity_dist + EntityEffects + TimeEffects'
mod_fe = PanelOLS.from_formula(formula_fe, data=polity_data)
res_fe = mod_fe.fit(cov_type="clustered", cluster_entity=True)
res_fe.summary

### 3.2 Anderson-Hsiao
---

In [ ]:
# Anderson-Hsiao
# Following Rachel Meager's code, let's use 2-step efficient GMM to fit the A.H. model

formula_ah = f'flow ~ {gdp_lags} + [polity_dist ~ polity_dist_1] + distw'
mod_ah = IVGMM.from_formula(formula_ah, data=polity_data)
res_ah = mod_ah.fit()
res_ah.summary

### 3.3 Arellano-Bond
---

In [ ]:
# Arellano-Bond
# Following Rachel Meager's code, let's use 2-step efficient GMM to fit the A.B. model
arellano_results = []
for i in range(2, n_lags + 1):
    polity_lags = '+'.join([f'polity_dist_{j}' for j in range(1, i+1)])
    formula_ab = f'flow ~ {gdp_lags} + [polity_dist ~ {polity_lags}] + distw'
    mod_ab = IVGMM.from_formula(formula_ab, data=polity_data)
    res_ab = mod_ab.fit()
    arellano_results.append(res_ab)

## 4. Comments
---